In [38]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import sqlite3
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("A SIMPLE MESSAGE BANK", style={'text-align': 'center'}),
    html.Div([
        html.Div("Submit", style={'font-size': '24px', 'margin-top': '20px'}),
        dcc.Input(id='input-handle', type='text', placeholder='Your Name or Handle:', style={'width': '300px', 'margin': '10px'}),
        dcc.Textarea(id='input-message', placeholder='Your Message:', style={'width': '300px', 'height': '100px', 'margin': '10px'}),
        html.Button('Submit', id='submit-button', n_clicks=0, style={'margin': '10px'}),
        html.Div(id='response-message', style={'margin': '10px'})
    ], style={'padding': '20px', 'border': '1px solid black'}),
    html.Div([
        html.Div("View", style={'font-size': '24px', 'margin-top': '20px'}),
        html.Button('Update', id='update-button', n_clicks=0, style={'margin': '10px'}),
        html.Div(id='display-messages', style={'margin': '10px'})
    ], style={'padding': '20px', 'border': '1px solid black', 'margin-top': '20px'})
], style={'width': '500px', 'margin': 'auto'})

# Function to get or create database connection
def get_message_db():
    db = sqlite3.connect('messages_db.sqlite', check_same_thread=False)
    cursor = db.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS messages
                      (handle TEXT, message TEXT)''')
    db.commit()
    return db

# Callback for submitting messages
@app.callback(
    Output('response-message', 'children'),
    Input('submit-button', 'n_clicks'),
    [State('input-handle', 'value'), State('input-message', 'value')],
    prevent_initial_call=True
)
def submit_message(n_clicks, handle, message):
    if handle and message:
        db = get_message_db()
        cursor = db.cursor()
        cursor.execute('INSERT INTO messages (handle, message) VALUES (?, ?)', (handle, message))
        db.commit()
        db.close()
        return 'Thank you! Your message has been submitted.'
    return 'Please fill in both your name and a message.'

# Function to fetch random messages
def random_messages(n=5):
    db = get_message_db()
    query = "SELECT handle, message FROM messages ORDER BY RANDOM() LIMIT ?"
    df = pd.read_sql_query(query, db, params=(n,))
    db.close()
    return df

# Callback for viewing messages
@app.callback(
    Output('display-messages', 'children'),
    Input('update-button', 'n_clicks'),
    prevent_initial_call=True
)
def view(n_clicks):
    df = random_messages(5)  # Fetch up to 5 random messages
    if df.empty:
        return "No messages to display."
    messages = [html.Li(f"{row['handle']}: {row['message']}") for index, row in df.iterrows()]
    return html.Ul(messages)

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
